In [ ]:
import edge_prediction_model

In [ ]:
import torch
import torch.nn.functional as F
import dgl

import numpy as np
import networkx as nx
import pathlib
import pickle
import random
import tqdm.auto as tqdm
import multiprocessing as mp

from sklearn.metrics import f1_score, accuracy_score
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = edge_prediction_model.Net(19, 256, 2, 8, F.relu)
model.load_state_dict(torch.load('../models/net_256_8_relu_0.0005_0.99_64_100.bin'))
model.to(device)
model.eval()

In [ ]:
def beam_search(g):
    depot = next(iter(g.nodes))
    solution = [depot]

    while len(solution) < len(g.nodes):
        i = solution[-1]
        neighbours = [(j, g.edges[(i, j)]['y_prob']) for j in g.neighbors(i) if j not in solution]

        nodes, p = zip(*neighbours)

        j = np.random.choice(nodes, p=np.array(p)/np.sum(p))

        solution.append(j)

    solution.append(depot)
    return solution

def solution_cost(g, solution):
    c = 0
    for i, j in zip(solution[:-1], solution[1:]):
        d = np.linalg.norm(g.nodes[j]['pos'] - g.nodes[i]['pos'])
        c += d
    return c

In [ ]:
data_dir = pathlib.Path('/local/scratch/bh511/data/150000_instances')

test_set = [l.strip() for l in open(data_dir / 'test.txt')]
scaler = pickle.load(open(data_dir / 'scaler.pkl', 'rb'))

In [ ]:
pool = mp.Pool(processes=None)

gaps = []
for test_path in tqdm.tqdm(test_set):
    g = nx.read_gpickle(test_path)
    lg = nx.line_graph(g)

    features = {e: scaler.transform(g.edges[e]['x'][np.newaxis, :]).squeeze() for e in lg.nodes}
    labels = {e: g.edges[e]['y'] for e in lg.nodes}
    edges = {e: e for e in lg.nodes}
    nx.set_node_attributes(lg, features, 'x')
    nx.set_node_attributes(lg, labels, 'y')
    nx.set_node_attributes(lg, edges, 'e')

    h = dgl.from_networkx(lg, node_attrs=['x', 'y', 'e'])
    h = h.to(device)
    x = h.ndata['x']
    y = h.ndata['y']
    e = h.ndata['e']

    with torch.no_grad():
        y_pred = model(h, x)
        y_prob = F.softmax(y_pred, dim=1)

#     in_solution = {e: float(g.edges[e]['y'][0]) for e in g.edges}
    p_in_solution = {tuple(k): v[1] for k, v in zip(e.cpu().numpy(), y_prob.cpu().numpy())}
    nx.set_edge_attributes(g, p_in_solution, 'y_prob')

    optimal_cost = 0
    for e in g.edges:
        if g.edges[e]['y'].squeeze() == 1:
            d = np.linalg.norm(g.nodes[e[1]]['pos'] - g.nodes[e[0]]['pos'])
            optimal_cost += d

    best_solution = None
    best_cost = 0

    def beam_search_pool(g, n):
        for i in range(n):
            yield g

    for solution in pool.imap(beam_search, beam_search_pool(g, 100)):
        cost = solution_cost(g, solution)
        if best_solution is None or cost < best_cost:
            best_solution = solution
            best_cost = cost

    gap = 100*(best_cost - optimal_cost)/optimal_cost
    gaps.append(gap)

In [ ]:
g = nx.read_gpickle(test_set[np.argmin(gaps)])
lg = nx.line_graph(g)

features = {e: scaler.transform(g.edges[e]['x'][np.newaxis, :]).squeeze() for e in lg.nodes}
labels = {e: g.edges[e]['y'] for e in lg.nodes}
edges = {e: e for e in lg.nodes}
nx.set_node_attributes(lg, features, 'x')
nx.set_node_attributes(lg, labels, 'y')
nx.set_node_attributes(lg, edges, 'e')

h = dgl.from_networkx(lg, node_attrs=['x', 'y', 'e'])
h = h.to(device)
x = h.ndata['x']
y = h.ndata['y']
e = h.ndata['e']

with torch.no_grad():
    y_pred = model(h, x)
    y_prob = F.softmax(y_pred, dim=1)

p_in_solution = {tuple(k): v[1] for k, v in zip(e.cpu().numpy(), y_prob.cpu().numpy())}
nx.set_edge_attributes(g, p_in_solution, 'y_prob')

optimal_cost = 0
for e in g.edges:
    if g.edges[e]['y'].squeeze() == 1:
        d = np.linalg.norm(g.nodes[e[1]]['pos'] - g.nodes[e[0]]['pos'])
        optimal_cost += d

best_solution = None
best_cost = 0

def beam_search_pool(g, n):
    for i in range(n):
        yield g

for solution in pool.imap(beam_search, beam_search_pool(g, 100)):
    cost = solution_cost(g, solution)
    if best_solution is None or cost < best_cost:
        best_solution = solution
        best_cost = cost
        
gap = 100*(best_cost - optimal_cost)/optimal_cost

In [ ]:
solution_edges = list(zip(best_solution[:-1], best_solution[1:]))
new_solution = {e: float(e in solution_edges or tuple(reversed(e)) in solution_edges) for e in g.edges}

In [ ]:
in_solution = {e: float(g.edges[e]['y'].squeeze()) for e in g.edges}

In [ ]:
cmap_colors = np.zeros((100, 4))
cmap_colors[:, 0] = 1
cmap_colors[:, 3] = np.linspace(0, 1, 100)
cmap = ListedColormap(cmap_colors)

pos = nx.get_node_attributes(g, 'pos')

fig, ax = plt.subplots(1, 3, figsize=(15, 5))

nx.draw(g, pos, edge_color=in_solution.values(), edge_cmap=cmap, ax=ax[0], edge_vmax=1, edge_vmin=0)
nx.draw(g, pos, edge_color=p_in_solution.values(), edge_cmap=cmap, ax=ax[1], edge_vmax=1, edge_vmin=0)
nx.draw(g, pos, edge_color=new_solution.values(), edge_cmap=cmap, ax=ax[2], edge_vmax=1, edge_vmin=0)
ax[0].set_title(f'Optimal Solution (cost={optimal_cost:.3f})')
ax[1].set_title('Edge likelihoods')
ax[2].set_title(f'Reconstructed Solution (cost={best_cost:.3f}, gap={gap:.3f}%)')

In [ ]:
np.mean(gaps)

In [ ]:
test_set[np.argmax(gaps)]